In [5]:
from random import randint


def roll(*, sides: int = 6, target: int = 4) -> bool:
    return randint(1, sides) >= target


def roll_dice(*, dice: int = 1, sides: int = 6, target: int = 4) -> int:
    return sum(roll(sides=sides, target=target) for _ in range(dice))

In [17]:
from collections import defaultdict
from dataclasses import dataclass


@dataclass(frozen=True)
class Test:
    dice: int
    target: int


@dataclass
class Result:
    successes: int = 0
    failures: int = 0

    @property
    def total(self) -> int:
        return self.successes + self.failures

    def success_percentage(self) -> float:
        return self.successes / self.total


max_dice = 6
dices = [i+1 for i in range(max_dice)]

max_target = 3
targets = [i+1 for i in range(max_target)]


test_results: dict[Test, Result] = defaultdict(Result)

for dice in dices:
    for target in targets:
        for _ in range(500_000):
            if roll_dice(dice=dice) >= target:
                test_results[Test(dice, target)].successes += 1
            else:
                test_results[Test(dice, target)].failures += 1

In [23]:
import pandas as pd

data = []
for dice in dices:
    data.append([dice, *(f"{test_results[Test(dice, target)].success_percentage() * 100:.1f}%" for target in targets)])

pd.DataFrame(data, columns=["D", *(f"Ob. {i}" for i in targets)])

,D,Ob. 1,Ob. 2,Ob. 3
0,1,49.9%,0.0%,0.0%
1,2,75.0%,25.1%,0.0%
2,3,87.4%,50.0%,12.6%
3,4,93.7%,68.8%,31.2%
4,5,96.9%,81.2%,49.9%
5,6,98.4%,89.1%,65.6%
